In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! nvidia-smi

/bin/bash: nvidia-smi: command not found


# **Install Dependencies**

In [ ]:
%%capture
%pip install tensorflow
%pip install keras
%pip install netron
%pip install nltk

#%pip3 install git+https://github.com/huggingface/transformers
%pip install datasets
%pip install nlp
%pip install rouge-score
!pip install sacrebleu
%pip install git-python
%pip install sentencepiece
!pip install torch
!pip install transformers
%pip install tree-sitter
%pip install evaluate
%pip install datasets
%pip install sentencepiece
%pip install rouge_score
%pip install sacremoses
%pip install torchdata
%pip install gradio
%pip install evaluate
%pip install tensorboard
%pip install netron
!pip install accelerate

# **Import Libraries**

In [ ]:
import transformers
import datasets
from datasets import load_dataset, load_metric, Dataset
import logging
from transformers import BertTokenizer, GPT2Tokenizer, GPT2TokenizerFast, EncoderDecoderModel, Trainer, TrainingArguments, BertTokenizerFast
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd
logging.basicConfig(level=logging.INFO)
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, EncoderDecoderModel
import types
import argparse
import logging
from functools import partial
import json

import torch
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    BertGenerationConfig,
    BertGenerationEncoder,
    BertTokenizer,
    EncoderDecoderModel,
    EncoderDecoderConfig,
    GPT2LMHeadModel,
    GPT2TokenizerFast,
    Trainer,
    TrainingArguments,
    get_cosine_schedule_with_warmup,
)

import sacrebleu
import random
import numpy as np
#from tensorboardX import SummaryWriter


# **Clear cache**

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cpu')

In [ ]:
%pwd

'/content'

# **Fetch dataset**

In [ ]:
!wget https://raw.githubusercontent.com/aub-mind/Arabic-Empathetic-Chatbot/master/arabic-empathetic-conversations.csv

--2023-06-16 22:23:54--  https://raw.githubusercontent.com/aub-mind/Arabic-Empathetic-Chatbot/master/arabic-empathetic-conversations.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7873052 (7.5M) [text/plain]
Saving to: ‘arabic-empathetic-conversations.csv’

arabic-empathetic-c 100%[===================>]   7.51M  --.-KB/s    in 0.04s   

2023-06-16 22:23:55 (207 MB/s) - ‘arabic-empathetic-conversations.csv’ saved [7873052/7873052]



In [ ]:
all_data = load_dataset("/content/Arabic-Empathetic-Chatbot/model/ArabicEmpatheticDialogues.py")


# all_data = load_dataset('csv', data_files={"/content/output_file.csv"},delimiter=',', quotechar= '"')

train_dataset = all_data['train'].train_test_split(test_size=0.2,seed=42)['train']
val_data = all_data['train'].train_test_split(test_size=0.2,seed=42)['test']
val_dataset = val_data.train_test_split(test_size=0.5,seed=42)['train']
test_dataset = val_data.train_test_split(test_size=0.5,seed=42)['test']


print("Length of train data",len(train_dataset))
print("Length of dev data",len(val_dataset))
print("Length of test data",len(test_dataset))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-489f75ffdf726d99/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Length of train data 2668
Length of dev data 334
Length of test data 334


In [ ]:
from prettytable import PrettyTable

# Select the first 5 records from the dataset
subset = train_dataset.select(range(15))

# Define the table headers
table = PrettyTable()
table.field_names = ["ID", "Question", "Answer"]

# Add each record to the table
for i in range(len(subset)):
    table.add_row([
        subset[i]["ID"],
        subset[i]["Question"],
         subset[i]["Answer"]
    ])


# Print the table
print(table)





+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
train_dataset

Dataset({
    features: ['ID', 'Question', 'Answer'],
    num_rows: 2668
})

In [ ]:
val_dataset

Dataset({
    features: ['ID', 'Question', 'Answer'],
    num_rows: 334
})

In [ ]:
test_dataset

Dataset({
    features: ['ID', 'Question', 'Answer'],
    num_rows: 334
})

# **Load Model and Tokenizer**

In [ ]:

model_name = "asafaya/bert-base-arabic"
model_name2 = "akhooli/gpt2-small-arabic"


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

In [ ]:

#model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = EncoderDecoderModel.from_encoder_decoder_pretrained(model_name, model_name2)
#model.to("cuda")


Some weights of the model checkpoint at asafaya/bert-base-arabic were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at akhooli/gpt2-small-arabic and are newly initialized: ['transformer.h.8.ln_cross_attn.bias', 'transformer.h.0.crossattention.c_proj.bias', 'transformer.h.1.crossattention.c_attn.weight', 'transformer.h.11.crossattention.q_attn.weight', 'transformer.h.1.ln_cross_attn.bias', 'transformer.h.2.ln_cross_attn.weight', 'transformer.h.7.crossattention.c_attn.weight', 'transformer.h.0.crossattention.q_attn.bias', 'transformer.h.7.crossattention.q_attn.bias', 'transformer.h.5.crossattention.c_attn.weight', 'transformer.h.3.crossattention.q_attn.weight', 'transformer.h.8.ln_cross_attn.weight', 'transformer.h.0.crossattention.c_proj.weight', 'transformer.h.1.ln_cross_attn.weight', 'transformer.h.4.crossattention.c_proj.bias', 'transformer.h.5.crossattention.q_attn.weight', 'transformer.h.3.crossattention.q_attn.bias', 'transformer.h.5.ln_cross_attn.bias', 'transformer.h.8.crossattention.c_attn.bias', 'transformer.h.8.

In [ ]:
model

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
#set special tokens
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id

#sensible parameters for beam search
#set decoding params
model.config.max_length = 128
model.config.early_stopping = True

model.config.num_beams = 1
model.config.vocab_size = model.config.encoder.vocab_size


In [ ]:
model.base_model.config.encoder.num_hidden_layers = 6
model.base_model.config.decoder.num_hidden_layers = 6

In [ ]:
model.tie_weights = True

# **Save model**

In [ ]:
model.save_pretrained("bert2gpt-model")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("./bert2gpt-model")
model

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_errors.py:259 in                  │
│ hf_raise_for_status                                                                              │
│                                                                                                  │
│   256 │   </Tip>                                                                                 │
│   257 │   """                                                                                    │
│   258 │   try:                                                                                   │
│ ❱ 259 │   │   response.raise_for_status()                                                        │
│   260 │   except HTTPError as e:                                                                 │
│   261 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   262                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/requests/models.py:960 in raise_for_status               │
│                                                                                                  │
│   957 │   │   │   http_error_msg = u'%s Server Error: %s for url: %s' % (self.status_code, rea   │
│   958 │   │                                                                                      │
│   959 │   │   if http_error_msg:                                                                 │
│ ❱ 960 │   │   │   raise HTTPError(http_error_msg, response=self)                                 │
│   961 │                                                                                          │
│   962 │   def close(self):                                                                       │
│   963 │   │   """Releases the connection back to the pool. Once this method has been             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/bert2gpt-model/resolve/main/config.json

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:417 in cached_file             │
│                                                                                                  │
│    414 │   user_agent = http_user_agent(user_agent)                                              │
│    415 │   try:                                                                                  │
│    416 │   │   # Load from URL or cache if already cached                                        │
│ ❱  417 │   │   resolved_file = hf_hub_download(                                                  │
│    418 │   │   │   path_or_repo_id,                                                              │
│    419 │   │   │   filename,                                                                     │
│    420 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:118 in _inner_fn    │
│                                                                                                  │
│   115 │   │   if check_use_auth_token:                                                           │
│   116 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   117 │   │                                                                 

# **Evaluator**

In [ ]:
from sacrebleu import corpus_bleu
def compute_metrics(pred):
  labels_ids = pred.label_ids
  #pred_ids = torch.argmax(pred.predictions,dim=2)
  pred_ids = pred.predictions

  # all unnecessary tokens are removed
  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  labels_ids[labels_ids == -100] = tokenizer.pad_token_id
  label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

  return {"bleu": round(corpus_bleu(pred_str , [label_str]).score, 4)}





# **Prepare the data**

In [ ]:
model.config.max_length

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [ ]:
encoder_length = 128
decoder_length = 128
batch_size = 8


# map data correctly
def map_to_encoder_decoder_inputs(batch):
    inputs = tokenizer(batch["Question"], padding="max_length", truncation=True, max_length=encoder_length)
    outputs = tokenizer(batch["Answer"], padding="max_length", truncation=True, max_length=decoder_length)

    #input_ids = inputs.input_ids.to("cuda")
    #attention_mask = inputs.attention_mask.to("cuda")

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask

    """
    # complicated list comprehension here because pad_token_id alone is not good enough to know whether label should be excluded or not
    batch["labels"] = [
        [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
    ]
    """
    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])

    return batch

In [ ]:
# make train dataset ready
train_data = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=["ID","Question", "Answer"])

train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Map:   0%|          | 0/2668 [00:00<?, ? examples/s]

In [ ]:
# same for validation dataset
val_data = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=["ID","Question", "Answer"],
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

Map:   0%|          | 0/334 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       max_length=128 , padding=True,
                                       model = model)

# **Training**

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./bert2gpt-Training",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    num_train_epochs=8,
    do_train=True,
    do_eval=True,
    fp16=True,
    overwrite_output_dir=True,
    learning_rate = 1e-5,
    weight_decay=0.01,
    warmup_ratio = 0.05,
    seed = 1995,
    save_total_limit = 2,
    load_best_model_at_end = True,
    #push_to_hub=True,
    optim = "adamw_hf",
)
#

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator = data_collator,
    train_dataset=train_data,
    eval_dataset =val_data,

)
#    save_total_limit=3,
#

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│ in __init__:116                                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1344 in __post_init__      │
│                                                                                                  │
│   1341 │   │   │   and (get_xla_device_type(self.device) != "GPU")                               │
│   1342 │   │   │   and (self.fp16 or self.fp16_full_eval)                                        │
│   1343 │   │   ):                                                                                │
│ ❱ 1344 │   │   │   raise ValueError(                                                             │
│   1345 │   │   │   │   "FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half  │
│   1346 │   │   │   │   " (`--fp16_full_eval`) can only be used on CUDA devices."                 │
│   1347 │   │   │   )                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation 
(`--fp16_full_eval`) can only be used on CUDA devices.

In [ ]:
#Training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the los

Epoch,Training Loss,Validation Loss,Bleu
1,0.660800,0.638561,0.476000
2,0.601700,0.590035,1.363400
3,0.562900,0.561020,1.252600
4,0.526900,0.542501,1.035000
5,0.510500,0.532600,1.294500
6,0.502000,0.525711,0.718400
7,0.495700,0.521368,1.287100


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a mo

In [ ]:
trainer.save_model()
trainer.save_state()

# **Copy to Google Drive**

In [ ]:
import shutil

# Define the source and destination folders
src_folder = '/content/bert2gpt-Training'
dst_folder = '/content/drive/MyDrive/bert2gpt-Training'

# Copy the folder and its contents to Google Drive
shutil.copytree(src_folder, dst_folder)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 8>:8                                                                              │
│                                                                                                  │
│ /usr/lib/python3.10/shutil.py:557 in copytree                                                    │
│                                                                                                  │
│    554 │   `src` tree.                                                                           │
│    555 │   """                                                                                   │
│    556 │   sys.audit("shutil.copytree", src, dst)                                                │
│ ❱  557 │   with os.scandir(src) as itr:                                                          │
│    558 │   │   entries = list(itr)                                                               │
│    559 │   return _copytree(entries=entries, src=src, dst=dst, symlinks=symlinks,                │
│    560 │   │   │   │   │    ignore=ignore, copy_function=copy_function,                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: '/content/bert2gpt-Training'

# **Visualize**

In [ ]:
%load_ext tensorboard

In [ ]:
#%reload_ext tensorboard

In [ ]:
pwd

In [ ]:
tensorboard --logdir ./bert2gpt-Training/runs/

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ tensorboard --logdir ./bert2gpt-Training/runs/                                                   │
│                       ▲                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax

# **Inference and Testing**

In [ ]:
#model_name = "bert2gpt-Training"
model_name = "/content/drive/MyDrive/bert2gpt-Training"

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
new_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
new_model.to("cuda")
new_tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
# map data correctly
def generate_response(context):
    inputs = new_tokenizer([context], padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = new_model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_response = new_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return output_response

In [ ]:
input_context = "كيف حالهم؟ هل تبلي بلاء حسنا في هذا المجال؟"
output_response = generate_response(input_context)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (128) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
print(output_response)

نعم ، لقد على الاقل.


In [ ]:
input_context = 'لا أعتقد أنني أريد الأطفال'
output_response = generate_response(input_context)

In [ ]:
print(output_response)

انا اسف لسماع ذلك.


In [ ]:
import gradio as gr

iface = gr.Interface(fn=generate_response, inputs="text", outputs="text",
                     examples=[["رأيت رجل بلا مأوى يتصبب عرقا في الشمس ذلك اليوم"],
                               ["هوجمت في الشارع من قبل شخص سرق حقيبتي. كنت خائفة جدًا لدرجة أنني أصبت بنوبة هلع."],
                               ["نعم ، لقد فعلت ولكني ما زلت مصدومة."],
                               ["عندما ولدت ابنتي كنت سعيدة ومتحمسة للغاية. "],
                                 ],
                     title="Arabic Islamic Question Answering",
                     description="This is a solution for question answering task")
iface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# **Push model to huggingface**

In [ ]:
!huggingface-cli login

In [ ]:
new_model.push_to_hub("AhmedSSoliman/Arabic-Islamic-Question-Answering")

In [ ]:
new_tokenizer.push_to_hub("AhmedSSoliman/Arabic-Islamic-Question-Answering")

# **Old_Code**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Install dependencies**

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:

%%capture
!pip install transformers
!pip install huggingface_hub
!pip install datasets
!pip install arabert
!pip install pyarabic
!pip install farasapy
!pip install numby
!pip install gradio
!pip install sacrebleu
!pip install rouge_score
!pip install git-python
!pip install git+https://github.com/huggingface/accelerate

# **Import Libraries**

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import os
import numpy as np
import pandas as pd
from datasets import load_dataset
import transformers
from transformers import BertTokenizer, EncoderDecoderModel
from sacrebleu import corpus_bleu
from transformers import BertTokenizerFast
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, TrainingArguments
from dataclasses import dataclass, field
from typing import Optional
import torch
import torch.nn as nn

In [ ]:
from transformers import EncoderDecoderModel, AutoTokenizer
from datasets import load_dataset
from arabert.preprocess import ArabertPreprocessor
from torch.utils.data.dataloader import DataLoader
from transformers import default_data_collator
from torch.utils.data.sampler import SequentialSampler
import torch
from tqdm.notebook import tqdm
import gradio as gr

# **Fetch dataset**

In [ ]:
!wget https://raw.githubusercontent.com/aub-mind/Arabic-Empathetic-Chatbot/master/arabic-empathetic-conversations.csv

--2023-05-20 20:46:35--  https://raw.githubusercontent.com/aub-mind/Arabic-Empathetic-Chatbot/master/arabic-empathetic-conversations.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7873052 (7.5M) [text/plain]
Saving to: ‘arabic-empathetic-conversations.csv’

arabic-empathetic-c 100%[===================>]   7.51M  --.-KB/s    in 0.08s   

2023-05-20 20:46:35 (90.4 MB/s) - ‘arabic-empathetic-conversations.csv’ saved [7873052/7873052]



In [ ]:

encoder_max_length = 32
decoder_max_length =32

model_name = "aubmindlab/bert-base-arabert"

tokenizer = BertTokenizerFast.from_pretrained(model_name)

In [ ]:
# all_data = load_dataset("/content/Arabic-Empathetic-Chatbot/model/ArabicEmpatheticDialogues.py")


all_data = load_dataset('csv', data_files={"/content/arabic-empathetic-conversations.csv"},delimiter=',', quotechar= '"')

train_data = all_data['train'].train_test_split(test_size=0.2,seed=42)['train']
val_data = all_data['train'].train_test_split(test_size=0.2,seed=42)['test']
dev_data = val_data.train_test_split(test_size=0.5,seed=42)['train']
test_data = val_data.train_test_split(test_size=0.5,seed=42)['test']


print("Length of train data",len(train_data))
print("Length of dev data",len(dev_data))
print("Length of test data",len(test_data))

  0%|          | 0/1 [00:00<?, ?it/s]

Length of train data 29302
Length of dev data 3663
Length of test data 3663


In [ ]:
# all_data = load_dataset('csv', data_files={"/content/arabic-empathetic-conversations.csv"},delimiter=',', quotechar= '"')

# # تحميل السجلات الأولى 1000 في مجموعة البيانات التدريبية
# train_data = all_data['train'].shuffle(seed=42).select(range(1000))
# # val_data = all_data['train'].select(range(1000, 1500))
# # تحميل السجلات الأولى 1000 في مجموعة البيانات التطويرية
# dev_data = all_data['train'].shuffle(seed=42).select(range(1000, 2000))

# # تحميل السجلات الأولى 1000 في مجموعة البيانات الاختبارية
# # test_data = all_data['test'].shuffle(seed=42).select(range(1000))
# print("Length of train data",len(train_data))
# print("Length of dev data",len(dev_data))
# print("Length of test data",len(test_data))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-870a57172e5188a2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Length of train data 1000
Length of dev data 1000


NameError: ignored

In [ ]:
train_data

In [ ]:
train_data[5]

In [ ]:

def process_data_to_model_inputs(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["context"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["response"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    # mask loss for padding
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
    ]
    batch["decoder_attention_mask"] = outputs.attention_mask

    return batch

In [ ]:

batch_size = 8


train_data = train_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["emotion", "context", "response"],
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

dev_data = dev_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["emotion","context", "response"],
)
dev_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

test_data = test_data.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["emotion","context", "response"],
)
test_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


Map:   0%|          | 0/3663 [00:00<?, ? examples/s]

In [ ]:
train_data

# **Model**

In [ ]:
arabert2arabert = EncoderDecoderModel.from_encoder_decoder_pretrained(model_name, model_name)
#, tie_encoder_decoder=False

Some weights of the model checkpoint at asafaya/bert-base-arabic were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at akhooli/gpt2-small-arabic and are newly initialized: ['transformer.h.0.crossattention.q_attn.weight', 'transformer.h.4.crossattention.c_proj.bias', 'transformer.h.4.crossattention.c_attn.weight', 'transformer.h.6.crossattention.c_proj.weight', 'transformer.h.0.crossattention.c_attn.weight', 'transformer.h.10.crossattention.c_proj.bias', 'transformer.h.7.ln_cross_attn.weight', 'transformer.h.9.crossattention.q_attn.weight', 'transformer.h.3.crossattention.bias', 'transformer.h.0.crossattention.c_proj.bias', 'transformer.h.11.crossattention.masked_bias', 'transformer.h.3.crossattention.c_proj.bias', 'transformer.h.6.ln_cross_attn.weight', 'transformer.h.8.crossattention.q_attn.weight', 'transformer.h.10.crossattention.c_proj.weight', 'transformer.h.11.ln_cross_attn.weight', 'transformer.h.6.crossattention.c_proj.bias', 'transformer.h.7.crossattention.q_attn.weight', 'transformer.h.9.crossattention.masked_bi

In [ ]:
arabert2arabert

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
arabert2arabert.config

EncoderDecoderConfig {
  "_commit_hash": null,
  "decoder": {
    "_name_or_path": "akhooli/gpt2-small-arabic",
    "activation_function": "gelu_new",
    "add_cross_attention": true,
    "architectures": [
      "GPT2LMHeadModel"
    ],
    "attn_pdrop": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 50256,
    "chunk_size_feed_forward": 0,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "embd_pdrop": 0.1,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 50256,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,
    "is_decoder": true,
    "is_encoder_decoder": false,
    "label2id": {
      "LABEL_0": 0,
      "LABEL_1": 1
    },
    "l

In [ ]:
arabert2arabert.config.encoder.num_hidden_layers

12

In [ ]:
arabert2arabert.config

In [ ]:
arabert2arabert.base_model.num_hidden_layers = 6

In [ ]:
arabert2arabert.config.num_layers =6

In [ ]:
arabert2arabert.config.max_length

20

In [ ]:
#set special tokens
arabert2arabert.config.decoder_start_token_id = tokenizer.cls_token_id
arabert2arabert.config.eos_token_id = tokenizer.sep_token_id
arabert2arabert.config.pad_token_id = tokenizer.pad_token_id

#sensible parameters for beam search
#set decoding params
arabert2arabert.config.max_length = 128
arabert2arabert.config.early_stopping = True

arabert2arabert.config.num_beams = 1
arabert2arabert.config.vocab_size = arabert2arabert.config.encoder.vocab_size


In [ ]:
arabert2arabert

In [ ]:
arabert2arabert.num_parameters

In [ ]:
arabert2arabert.config

In [ ]:
arabert2arabert.base_model.config.encoder.num_hidden_layers = 6
arabert2arabert.base_model.config.decoder.num_hidden_layers = 6

In [ ]:
arabert2arabert.tie_weights = True

In [ ]:
arabert2arabert

# **Save model**

In [ ]:
arabert2arabert.save_pretrained("arabert2arabert-model")

In [ ]:
from transformers import AutoModelForSeq2SeqLM
arabert2arabert = AutoModelForSeq2SeqLM.from_pretrained("./arabert2arabert-model")
arabert2arabert

Some weights of the model checkpoint at ./arabert2arabert-model were not used when initializing EncoderDecoderModel: ['encoder.encoder.layer.10.attention.self.value.weight', 'decoder.transformer.h.6.attn.bias', 'encoder.encoder.layer.11.attention.self.value.weight', 'decoder.transformer.h.9.ln_1.weight', 'decoder.transformer.h.10.ln_1.weight', 'encoder.encoder.layer.11.output.dense.bias', 'encoder.encoder.layer.7.attention.self.value.weight', 'encoder.encoder.layer.10.attention.output.LayerNorm.bias', 'encoder.encoder.layer.8.intermediate.dense.bias', 'decoder.transformer.h.6.mlp.c_proj.bias', 'decoder.transformer.h.8.crossattention.masked_bias', 'encoder.encoder.layer.7.attention.output.dense.weight', 'encoder.encoder.layer.8.intermediate.dense.weight', 'decoder.transformer.h.9.mlp.c_proj.bias', 'encoder.encoder.layer.8.output.dense.weight', 'decoder.transformer.h.6.mlp.c_proj.weight', 'decoder.transformer.h.11.ln_1.weight', 'encoder.encoder.layer.7.attention.self.key.bias', 'decoder.

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [ ]:
arabert2arabert.config

EncoderDecoderConfig {
  "_commit_hash": null,
  "_name_or_path": "./arabert2arabert-model",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "akhooli/gpt2-small-arabic",
    "activation_function": "gelu_new",
    "add_cross_attention": true,
    "architectures": [
      "GPT2LMHeadModel"
    ],
    "attn_pdrop": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 50256,
    "chunk_size_feed_forward": 0,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "embd_pdrop": 0.1,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 50256,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,
    "is_decoder": true,
    "

In [ ]:
def compute_metrics(pred):
  labels_ids = pred.label_ids
  #pred_ids = torch.argmax(pred.predictions,dim=2)
  pred_ids = pred.predictions

  # all unnecessary tokens are removed
  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  labels_ids[labels_ids == -100] = tokenizer.pad_token_id
  label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

  return {"bleu": round(corpus_bleu(pred_str , [label_str]).score, 4)}



In [ ]:
from transformers import Seq2SeqTrainingArguments

In [ ]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate


In [ ]:
!pip install --upgrade accelerate

In [ ]:
#Set training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./model",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps = 2,
    # predict_with_generate=True,
    do_eval=True,
    evaluation_strategy ="epoch",
    do_train=True,
    #logging_steps=500,
    #save_steps= 32965 // ( batch_size * 2),
    warmup_steps=100,
    eval_steps=10,
    # max_steps=16, # delete for full training
    num_train_epochs=3,# uncomment for full training
    overwrite_output_dir=True,
    save_total_limit=1,
    fp16=True,
    #generation_max_length=50
)



In [ ]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=arabert2arabert,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=dev_data,
)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Epoch,Training Loss,Validation Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1664 in train                    │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1940 in _inner_training_loop     │
│                                                                                                  │
│   1937 │   │   │   │   │   with model.no_sync():                                                 │
│   1938 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1939 │   │   │   │   else:                                                                     │
│ ❱ 1940 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1941 │   │   │   │                                                                             │
│   1942 │   │   │   │   if (                                                                      │
│   1943 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2745 in training_step            │
│                                                                                                  │
│   2742 │   │   │   loss = loss / self.args.gradient_accumulation_steps                           │
│   2743 │   │                                                                                     │
│   2744 │   │   if self.do_grad_scaling:                                                          │
│ ❱ 2745 │   │   │   self.scaler.scale(loss).backward()                                            │
│   2746 │   │   elif self.use_apex:                                                               │
│   2747 │   │   │   with amp.scale_loss(loss, self.optimizer) as scaled_loss:                     │
│   2748 │   │   │   │   scaled_loss.backward()                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/_tensor.py:487 in backward                         │
│                                                                                                  │
│    484 │   │   │   │   create_graph=create_graph,                                                │
│    485 │   │   │   │   inputs=inputs,                                                            │
│    486 │   │   │   )                                                                             │
│ ❱  487 │   │   torch.autograd.backward(                                                          │
│    488 │   │   │   self, gradient, retain_graph, create_gra

In [ ]:
#Save tokenizer and model
trainer._save("./arabert2arabert1")
tokenizer.save_pretrained("./arabert2arabert1")


In [ ]:
#Evaluate
eval_output = trainer.evaluate()

In [ ]:
#Compute perplexity
import math
perplexity = math.exp(eval_output["eval_loss"])
print('\nEvaluate Perplexity: {:10,.2f}'.format(perplexity))

In [ ]:
model_name="bert-base-arabert"
arabert_prep = ArabertPreprocessor(model_name=model_name, keep_emojis=False)


tokenizer = AutoTokenizer.from_pretrained("./arabert2arabert")
model = EncoderDecoderModel.from_pretrained("./arabert2arabert")

model.to("cuda")
model.eval()
print("done")

In [ ]:
def generate_response(text, minimum_length, k):
  text_clean = arabert_prep.preprocess(text)
  inputs = tokenizer.encode_plus(text_clean,return_tensors='pt')
  outputs = model.generate(input_ids = inputs.input_ids.to("cuda"),
                   attention_mask = inputs.attention_mask.to("cuda"),
                   num_beams=1,
                   do_sample = True,
                   min_length=minimum_length,
                   top_k = k,
                   temperature = 1,
                   length_penalty =2)
  preds = tokenizer.batch_decode(outputs)
  response = str(preds)
  response = response.replace("\'", '')
  response = response.replace("[[CLS]", '')
  response = response.replace("[SEP]]", '')
  response = str(arabert_prep.desegment(response))
  return response

In [ ]:
gr.Interface(fn=generate_response,
              inputs=[
          gr.inputs.Textbox(),
          gr.inputs.Slider(5, 20, step=1, label='Minimum Output Length'),
          gr.inputs.Slider(10, 1000, step=10, label='Top-K'),
          ],
             outputs="text").launch(share=True)